Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Нужно обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.



### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

In [ ]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer 
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import f1_score, confusion_matrix

import warnings
warnings.filterwarnings('ignore')


import numpy as np
import torch
import transformers
from tqdm import notebook

In [ ]:
df = pd.read_csv('/datasets/toxic_comments.csv')
df.head(5)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Напишем функции для лемматизации и очистки текстаю

In [ ]:
nltk.download('wordnet')

def lemmatize(text):
    m = WordNetLemmatizer()
    lemm_list = m.lemmatize(text)
    lemm_text = "".join(lemm_list)
        
    return lemm_text


def clear_text(text):
    re_list = re.sub(r"[^a-zA-Z']", ' ', text)
    re_list = re_list.split()
    re_list = " ".join(re_list)
    return re_list

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Создадим корпус из очищенных текстов.

In [ ]:
%%time
corpus = list(df['text'].apply(lambda x: lemmatize(clear_text(x))))

CPU times: user 10.2 s, sys: 270 ms, total: 10.5 s
Wall time: 10.5 s


Переведем тексты в вектора и создадим матрицу с оценками важности слов. Она будет использоваться в качестве features в наших моделях.

In [ ]:
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words = stopwords)
tf_idf = count_tf_idf.fit_transform(corpus)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
tf_idf.shape, df['toxic'].shape

((159571, 168645), (159571,))

# 2. Обучение

In [ ]:
features = tf_idf
target = df['toxic'].values


train_features, valid_feature, train_target, valid_target = train_test_split(features, 
                                                                           target, 
                                                                           test_size=0.2, 
                                                                           random_state=42)

valid_features, test_features, valid_target, test_target = train_test_split(valid_feature, 
                                                                           valid_target, 
                                                                           test_size=0.5,
                                                                           random_state=42)

In [ ]:
target

array([0, 0, 0, ..., 0, 0, 0])

Обучим несколько моделей.


In [ ]:
%%time

reg_model = LogisticRegression(class_weight = 'balanced')
reg_model.fit(train_features, train_target)
prediction = reg_model.predict(valid_features)
f1 = f1_score(valid_target, prediction)
print('F1 логистической регрессии:', f1)
print()
print('Матрица ошибок')
print(confusion_matrix(valid_target, prediction))
print()

F1 логистической регрессии: 0.7532608695652175

Матрица ошибок
[[13663   647]
 [  261  1386]]

CPU times: user 7.2 s, sys: 6.41 s, total: 13.6 s
Wall time: 13.6 s


In [ ]:
%%time

cat_model = CatBoostClassifier(eval_metric="F1", 
                                   iterations=100, 
                                   max_depth=6, 
                                   learning_rate=0.9, 
                                   random_state=42)
cat_model.fit(train_features, train_target, verbose=20)
prediction = cat_model.predict(valid_features)
f1 = f1_score(valid_target, prediction)
print('F1 CatBoost:', f1)
print()
print('Матрица ошибок')
print(confusion_matrix(valid_target, prediction))
print()

0:	learn: 0.4077145	total: 6.74s	remaining: 11m 7s
20:	learn: 0.7365763	total: 1m 59s	remaining: 7m 28s
40:	learn: 0.7697634	total: 3m 51s	remaining: 5m 33s
60:	learn: 0.7898848	total: 5m 44s	remaining: 3m 40s
80:	learn: 0.8029253	total: 7m 39s	remaining: 1m 47s
99:	learn: 0.8113825	total: 9m 26s	remaining: 0us
F1 CatBoost: 0.7601224906430758

Матрица ошибок
[[14135   175]
 [  530  1117]]

CPU times: user 11min 2s, sys: 16.7 s, total: 11min 19s
Wall time: 11min 26s


Лучший результат получается у CatBoost.

# 3. Выводы

Проверим модели на тестовой выборке

In [ ]:
prediction = cat_model.predict(test_features)
f1 = f1_score(test_target, prediction)
print('F1 CatBoost:', f1)
print()
print('Матрица ошибок')
print(confusion_matrix(test_target, prediction))
print()

F1 CatBoost: 0.7551954913702007

Матрица ошибок
[[14191   170]
 [  525  1072]]



In [ ]:
prediction = reg_model.predict(test_features)
f1 = f1_score(test_target, prediction)
print('F1 регрессии:', f1)
print()
print('Матрица ошибок')
print(confusion_matrix(test_target, prediction))
print()

F1 регрессии: 0.756179775280899

Матрица ошибок
[[13744   617]
 [  251  1346]]



#### Вывод
Обе модели имеют одинаковое значение метрики F1. Но, как мы видим из матрицы ошибок, у модели регрессии больше ложноположительных ответов и меньше ложно отрицательных. У CatBoost наоборот - больше ложно отрицательных и меньше ложноположительных. 
На мой взляд, для бизнасе будет полезнне модель, которая меньше ошибается в определении отрицательных комментариев, чем положительных. Потому что положительный комментарий можно промодерировать вручную и вернуть, а чем меньше мдель пропускает негативных комментариев, тем лучше.

Такрим образом, для бизнеса, лучше подходит модель **линейной регрессии**.

# 4. BERT

Попытка применить модель предобученную модель BERT. 

In [ ]:
df_comments = pd.read_csv('/datasets/toxic_comments.csv')
df_comments = df_comments.sample(400).reset_index(drop=True)

In [ ]:
tokenizer = transformers.BertTokenizer(vocab_file='vocab.txt')
tokenized = df_comments['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=400))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
        
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)

In [ ]:
config = transformers.BertConfig.from_json_file('bert_config.json')
model = transformers.BertModel.from_pretrained('pytorch_model.bin', config=config)

In [ ]:
"""batch_size = 40
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
    
    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)
    embeddings.append(batch_embeddings[0][:,0,:])"""

#Срок выполнения - 25 минут. 

In [ ]:
features_bert = np.concatenate(embeddings)
target_bert = df_comments['toxic']

train_features_bert, test_features_bert, train_target_bert, test_target_bert = train_test_split(features_bert, 
                                                                                                target_bert, 
                                                                                                test_size=0.2, 
                                                                                                random_state = 42)

bert_model = LogisticRegression(class_weight = 'balanced')
bert_model.fit(train_features_bert, train_target_bert)
prediction_bert = bert_model.predict(test_features_bert)
f1_bert = f1_score(test_target_bert, prediction_bert)
print('F1 BERT:', f1_bert)
print()
print('Матрица ошибок')
print(confusion_matrix(test_target_bert, prediction_bert))
print()


#F1 = 0.70
#Матрица ошибок
#[[69  2]
# [ 3  6]]

F1 BERT: 0.7058823529411765

Матрица ошибок
[[69  2]
 [ 3  6]]

